In [ ]:
# Data linked to this kaggle notebook - 2 arrays cantaining Binary seg masks and MRI t1_scans

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
file_paths=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_paths.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
from torch.nn import DataParallel
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,DataLoader,random_split
import tqdm.auto as tqdm
import shutil
from sklearn.metrics import jaccard_score  


In [3]:
class Conv_Block(nn.Module):
    def __init__(self,in_channels,num_filters):
        super().__init__()

        self.conv_block = nn.Sequential(
            nn.Conv3d(in_channels=in_channels, out_channels=num_filters, kernel_size=(3, 3, 3), padding=1),
            nn.BatchNorm3d(num_filters),
            nn.ReLU(),
            nn.Conv3d(in_channels=num_filters, out_channels=num_filters, kernel_size=(3, 3, 3), padding=1),
            nn.BatchNorm3d(num_filters),
            nn.ReLU()
        )

    def forward(self,x):
        return self.conv_block(x)

class encoder_block(nn.Module):
    def __init__(self,in_channels,num_filters):
        super().__init__()
        self.conv_block = Conv_Block(in_channels,num_filters)

    def forward(self,x):
        s = self.conv_block(x)
        p = nn.MaxPool3d(kernel_size = (2,2,2),stride = 2)(s)

        return s,p


class decoder_block(nn.Module):
    def __init__(self,in_channels,num_filters):
        super().__init__()

        self.up_conv = nn.ConvTranspose3d(in_channels=in_channels,out_channels=num_filters,kernel_size=(2,2,2),stride=2,padding=0)
        self.conv_block = Conv_Block(num_filters*2, num_filters)

    def forward(self,x,skip_con):
        x = self.up_conv(x)
        x = torch.cat([x,skip_con],dim = 1)
        x = self.conv_block(x)
        return x
    
class UNet3D(nn.Module):
    def __init__(self):
        super().__init__()

        
        
        self.enc1 = encoder_block(1,8)
        self.enc2 = encoder_block(8,16)
        self.enc3 = encoder_block(16,32)
        
        self.b1 = Conv_Block(32,64)

        self.dec1 = decoder_block(64,32)
        self.dec2 = decoder_block(32,16)
        self.dec3 = decoder_block(16,8)
        
        self.output = nn.Sequential(
            nn.Conv3d(in_channels = 8,out_channels=1,kernel_size = (1,1,1),padding = 0),
            nn.Sigmoid()                       
                                   )
    def forward(self,x):
        s1,p1 = self.enc1(x)
        s2,p2 = self.enc2(p1)
        s3,p3 = self.enc3(p2)

        b1 = self.b1(p3)

        d1 = self.dec1(b1,s3)
        d2 = self.dec2(d1,s2)
        d3 = self.dec3(d2,s1)

        outputs = self.output(d3)

        return outputs

        
        
        

In [4]:
file_paths

['/kaggle/input/brats-preprocessed/seg_masks_resized.npy',
 '/kaggle/input/brats-preprocessed/t1_ce_scans_resized.npy']

In [5]:
scans = torch.from_numpy(np.load(file_paths[1])).unsqueeze(1)
segments = torch.from_numpy(np.load(file_paths[0])).unsqueeze(1)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(scans,segments,test_size=0.2,random_state = 42)
train_dataset = TensorDataset(X_train,y_train)
test_dataset = TensorDataset(X_test,y_test)
train_loader = DataLoader(train_dataset, batch_size = 4)
test_loader = DataLoader(test_dataset,batch_size = 4)

In [10]:
X_train.shape

torch.Size([295, 1, 128, 128, 128])

# **Normal Training**

In [8]:
import torch
import numpy as np

model = DataParallel(UNet3D().cuda())
loss_fn = nn.BCELoss()  
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
epochs = 20

train_losses = []
val_losses = []
val_ious = []
best_iou = 0.0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for scans, masks in tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training", leave=False):
        scans = scans.cuda()
        masks = masks.cuda()

        optimizer.zero_grad()
        outputs = model(scans)  # Already has sigmoid applied

        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() 

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation 
    model.eval()
    val_loss = 0.0
    iou_scores = []

    with torch.no_grad():
        for scans, masks in tqdm.tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} - Testing", leave=False):  
            scans = scans.cuda()
            masks = masks.cuda()

            outputs = model(scans)  # Already sigmoid activated
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()

            # Thresholding predictions
            preds = (outputs > 0.5).cpu().numpy().astype(np.uint8)  
            masks_np = masks.cpu().numpy().astype(np.uint8)

            iou = jaccard_score(masks_np.flatten(), preds.flatten(), average='macro', zero_division=0)
            iou_scores.append(iou)

    avg_val_loss = val_loss / len(test_loader)  
    avg_iou = np.mean(iou_scores)

    val_losses.append(avg_val_loss)
    val_ious.append(avg_iou)
    if(avg_iou>best_iou):
        best_iou = avg_iou
        file_path = f'model_350k_{avg_iou:.4f}.pth'
        torch.save(model.state_dict(),file_path)
        shutil.move(file_path, f'/kaggle/working/' + file_path)

    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val IoU: {avg_iou:.4f}")


Epoch 1/20 - Training:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 1/20 - Testing:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/20]
Train Loss: 0.5951, Val Loss: 0.5733, Val IoU: 0.5086


Epoch 2/20 - Training:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 2/20 - Testing:   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 19/20 - Testing:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch [19/20]
Train Loss: 0.0372, Val Loss: 0.0527, Val IoU: 0.6608


Epoch 20/20 - Training:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 20/20 - Testing:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch [20/20]
Train Loss: 0.0347, Val Loss: 0.0424, Val IoU: 0.7038


# Split the training data into 3 Clients

In [7]:
C1_train_img = X_train[:99]
C2_train_img = X_train[99:197]
C3_train_img = X_train[197:]



C1_train_mask = y_train[:99]
C2_train_mask = y_train[99:197]
C3_train_mask = y_train[197:]


C1_train_ds = TensorDataset(C1_train_img,C1_train_mask)
C3_train_ds = TensorDataset(C3_train_img,C3_train_mask)
C2_train_ds = TensorDataset(C2_train_img,C2_train_mask)

C1_train_loader = DataLoader(C1_train_ds,batch_size = 4,shuffle = True)
C2_train_loader = DataLoader(C2_train_ds,batch_size = 4,shuffle = True)
C3_train_loader = DataLoader(C3_train_ds,batch_size = 4,shuffle = True)


In [8]:
def train_local_model(model, train_loader,loss_fn, epochs=1):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0015)
    for epoch in range(epochs):

        for scan, mask in tqdm.tqdm(train_loader,desc=f"Epoch {epoch+1}/{epochs} - Training", leave=False):
            scan, mask = scan.cuda(), mask.cuda()
           
            output = model(scan) 
            optimizer.zero_grad()
            loss = loss_fn(output, mask)
            loss.backward()
            optimizer.step()
    return model.state_dict()


def evaluate(model,test_loader,loss_fn):

    model.eval()
    val_loss = 0.0
    iou_scores = []

    with torch.inference_mode():
        for scan,mask in test_loader:
            scan, mask = scan.cuda(), mask.cuda()

            outputs = model(scan)
            val_loss+=loss_fn(outputs,mask)
            preds = (outputs > 0.5).cpu().numpy().astype(np.uint8)  
            masks_np = mask.cpu().numpy().astype(np.uint8)

            iou = jaccard_score(masks_np.flatten(), preds.flatten(), average='binary', zero_division=0)
            iou_scores.append(iou)
            
        avg_val_loss = val_loss / len(test_loader)  
        avg_iou = np.mean(iou_scores)

        print(f"Test Loss: {avg_val_loss:.3f}, Test IOU: {avg_iou:.3f}")
            
            
            

In [9]:
def FedAvg(global_model, client_models):
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i][k].float() for i in range(len(client_models))], dim=0).mean(0)
    global_model.load_state_dict(global_dict)
    return global_model

# Federated Learning Training Loop


In [10]:
global_model = DataParallel(UNet3D().cuda())
clients = [C1_train_loader, C2_train_loader, C3_train_loader]
num_clients = len(clients)
rounds = 30
loss_fn = nn.BCELoss()
for round_no in range(rounds):
    print(f"Round: {round_no+1}")
    client_models = []

    for train_loader in clients:
        local_model = DataParallel(UNet3D().cuda())
        local_model.load_state_dict(global_model.state_dict())
        client_state_dict = train_local_model(local_model,train_loader,loss_fn,7)
        client_models.append(client_state_dict)
        del local_model  # Free memory
        torch.cuda.empty_cache()  # Clear unused memory

    global_model = FedAvg(global_model, client_models)

    evaluate(global_model,test_loader,loss_fn)        

Round: 1


Epoch 1/7 - Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 2/7 - Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 3/7 - Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 4/7 - Training:   0%|          | 0/25 [00:00<?, ?it/s]

...

Epoch 4/7 - Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 5/7 - Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 6/7 - Training:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 7/7 - Training:   0%|          | 0/25 [00:00<?, ?it/s]

Test Loss: 0.017, Test IOU: 0.627
